In [1]:
## #
## # Preparacion de datos para el LAB:
## #
## # https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset
## #
##
##
## import pandas as pd  # type : ignore
##
## df = pd.read_csv("../files/input/used_cars.csv")
## train = df.sample(frac=0.7, random_state=1)
## test = df.drop(train.index)
##
## train.to_csv(
##     "../files/input/train_data.csv.zip",
##     index=False,
##     compression="zip",
## )
## test.to_csv(
##     "../files/input/test_data.csv.zip",
##     index=False,
##     compression="zip",
## )

In [2]:
#
# En este dataset se desea pronosticar el precio de vhiculos usados. El dataset
# original contiene las siguientes columnas:
#
# - brand: Marca del vehiculo.
# - model: Modelo del vehiculo.
# - model_year: Año de fabricación.
# - mileage: Millas recorridas.
# - fuel_type: Tipo de combustible (gasoline, diesel, electric, hybrid).
# - engine: Esepecificaciones del motor.
# - trainsmission: Tipo de transmisión (automatic, manual, other).
# - ext_col: Color exterior.
# - int_col: Color interior.
# - acccident: Historial de accidentes o daños.
# - clean_title: Título limpio (para efectos legales).
# - price: Precio del vehiculo.
#
# El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
# en la carpeta "files/input/".
#
# Los pasos que debe seguir para la construcción de un modelo de
# pronostico están descritos a continuación.
#

import pandas as pd  #  type: ignore

raw_train_df = pd.read_csv(
    "../files/input/train_data.csv.zip",
    compression="zip",
)

raw_test_df = pd.read_csv(
    "../files/input/test_data.csv.zip",
    compression="zip",
)

display(raw_train_df.head())
display(raw_test_df.head())

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Kia,Telluride SX,2020,"55,400 mi.",Gasoline,291.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Green,White,NaN,NaN,"$36,500"
1,Ford,Mustang GT,2010,"87,500 mi.",Gasoline,315.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,A/T,Red,Black,At least 1 accident or damage reported,Yes,"$14,900"
2,Ford,F-250 Lariat,2017,"72,128 mi.",E85 Flex Fuel,6.2L V8 16V MPFI SOHC Flexible Fuel,6-Speed Automatic,White,Camel,NaN,NaN,"$45,985"
3,Lexus,LX 600 Premium,2023,"1,500 mi.",Gasoline,409.0HP 3.4L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Gray,Black,NaN,NaN,"$119,500"
4,BMW,228 Gran Coupe i xDrive,2020,"10,426 mi.",Gasoline,2.0 Liter Turbo,Automatic,Black Sapphire Metallic,Oyster/Black,None reported,NaN,"$30,798"


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Acura,ILX 2.4L,2016,"136,397 mi.",Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,"$14,798"
1,Jaguar,F-TYPE,2020,"15,903 mi.",Gasoline,2.0 Liter Supercharged,Automatic,Silver,Black,None reported,NaN,"$47,998"
2,Land,Rover LR4 HSE,2013,"79,800 mi.",Gasoline,375.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,"$29,990"
3,Nissan,350Z Enthusiast,2003,"74,000 mi.",Gasoline,287.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed M/T,Purple,–,None reported,Yes,"$11,000"
4,Genesis,GV70 3.5T Sport,2023,"5,400 mi.",Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Green,Beige,NaN,NaN,"$60,000"


In [39]:
pprint(raw_train_df.columns.to_list())

['brand',
 'model',
 'model_year',
 'milage',
 'fuel_type',
 'engine',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title',
 'price']


In [41]:
from pprint import pprint

pprint(sorted(raw_train_df.clean_title.value_counts().index.to_list()))
display(raw_train_df.clean_title.isna().sum())
pprint(sorted(raw_test_df.clean_title.value_counts().index.to_list()))
display(raw_test_df.clean_title.isna().sum())

['Yes']


np.int64(422)

['Yes']


np.int64(174)

In [45]:
#
# Paso 1.
# Realice la limpieza de los datasets:
# - Procese la columna "price" para convertirla en valores numéricos.
# - Reemplace los valores desconocidos en la columna "fuel_type" por "unknown".
# - Convierta los valores de la columna "accident" a "reported" y "not_reported".
# -


#
#
def preprocess_data(df):

    df = df.copy()

    ## brand:
    df["brand"] = df["brand"].fillna("missing")
    df["brand"] = df["brand"].astype("category")

    ## is_luxury (calculated):
    luxury_brands = [
        "Mercedes-Benz",
        "BMW",
        "Audi",
        "Porsche",
        "Land",
        "Lexus",
        "Jaguar",
        "Bentley",
        "Maserati",
        "Lamborghini",
        "Rolls-Royce",
        "Ferrari",
        "McLaren",
        "Aston",
        "Maybach",
    ]
    df["is_luxury"] = df["brand"].apply(lambda x: x in luxury_brands)

    ## model:
    df["model"] = df["model"].fillna("missing")
    df["model"] = df["model"].astype("category")

    ## model_year:
    df["model_year"] = df["model_year"].astype("int")

    ## age (calculated):
    df["age"] = 2024 - df["model_year"]

    ## milage:
    df["milage"] = (
        df["milage"].str.replace(" mi.", "").str.replace(",", "").astype(float)
    )

    ## milage_per_year (calculated):
    df["milage_per_year"] = df["milage"] / df["age"]

    ## fuel_type: {'Gasoline', 'Hybrid', 'E85 Flex Fuel', 'Diesel', '–',
    ##  'Plug-In Hybrid', 'not supported'}
    df.loc[df["fuel_type"] == "–", "fuel_type"] = "missing"
    df["fuel_type"] = df["fuel_type"].fillna("missing")
    df["fuel_type"] = df["fuel_type"].astype("category")

    ## engine:
    df["engine"] = df["engine"].fillna("missing")
    df["engine"] = df["engine"].astype("category")

    ## horsepower (calculated):
    def extract_horsepower(x):
        x = x.split()
        x = [t for t in x if t.endswith("HP")]
        if len(x) == 0:
            return None
        return float(x[0].replace("HP", ""))

    df["horsepower"] = df["engine"].apply(extract_horsepower)

    ## engine_size (calculated):
    def extract_engine_size(x):
        x = x.split()
        x = [t for t in x if t.endswith("L")]
        if len(x) == 0:
            return None
        return float(x[0].replace("L", ""))

    df["engine_size"] = df["engine"].apply(extract_engine_size)

    ## horsepower_to_engine_size (calculated):
    df["horsepower_to_engine_size"] = df["horsepower"] / df["engine_size"]

    ## transmission:
    df["transmission"] = df["transmission"].fillna("missing")
    df["transmission"] = df["transmission"].astype("category")

    ## ext_col
    df["ext_col"] = df["ext_col"].fillna("missing")
    df["ext_col"] = df["ext_col"].astype("category")

    ## int_col:
    df["int_col"] = df["int_col"].fillna("missing")
    df["int_col"] = df["int_col"].astype("category")

    ## accdident: {NaN, 'None reported', 'At least 1 accident or damage reported'}
    df.loc[df["accident"].isna(), "accident"] = "not_reported"
    df.loc[df["accident"] == "None reported", "accident"] = "not_reported"
    df.loc[df["accident"] != "not_reported", "accident"] = "reported"

    ## clean_title
    df["clean_title"] = df["clean_title"].fillna("No")

    ## accident_impact (calculated):
    df["accident_impact"] = (df["accident"] == "reported") & (df["clean_title"] == "No")

    ## price: $14,798
    df.loc[:, "price"] = (
        df["price"].str.replace("$", "").str.replace(",", "").astype(float)
    )

    return df


cleaned_train_df = preprocess_data(raw_train_df)
cleaned_test_df = preprocess_data(raw_test_df)

display(cleaned_train_df.head())
display(cleaned_test_df.head())

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,is_luxury,age,milage_per_year,horsepower,engine_size,horsepower_to_engine_size,accident_impact
0,Kia,Telluride SX,2020,55400.0,Gasoline,291.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Green,White,not_reported,No,36500.0,False,4,13850.0,291.0,3.8,76.578947,False
1,Ford,Mustang GT,2010,87500.0,Gasoline,315.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,A/T,Red,Black,reported,Yes,14900.0,False,14,6250.0,315.0,4.6,68.478261,False
2,Ford,F-250 Lariat,2017,72128.0,E85 Flex Fuel,6.2L V8 16V MPFI SOHC Flexible Fuel,6-Speed Automatic,White,Camel,not_reported,No,45985.0,False,7,10304.0,NaN,6.2,NaN,False
3,Lexus,LX 600 Premium,2023,1500.0,Gasoline,409.0HP 3.4L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Gray,Black,not_reported,No,119500.0,True,1,1500.0,409.0,3.4,120.294118,False
4,BMW,228 Gran Coupe i xDrive,2020,10426.0,Gasoline,2.0 Liter Turbo,Automatic,Black Sapphire Metallic,Oyster/Black,not_reported,No,30798.0,True,4,2606.5,NaN,NaN,NaN,False


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,is_luxury,age,milage_per_year,horsepower,engine_size,horsepower_to_engine_size,accident_impact
0,Acura,ILX 2.4L,2016,136397.0,Gasoline,2.4 Liter,F,Silver,Ebony.,not_reported,No,14798.0,False,8,17049.625000,NaN,NaN,NaN,False
1,Jaguar,F-TYPE,2020,15903.0,Gasoline,2.0 Liter Supercharged,Automatic,Silver,Black,not_reported,No,47998.0,True,4,3975.750000,NaN,NaN,NaN,False
2,Land,Rover LR4 HSE,2013,79800.0,Gasoline,375.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,A/T,White,Black,not_reported,Yes,29990.0,True,11,7254.545455,375.0,5.0,75.000000,False
3,Nissan,350Z Enthusiast,2003,74000.0,Gasoline,287.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed M/T,Purple,–,not_reported,Yes,11000.0,False,21,3523.809524,287.0,3.5,82.000000,False
4,Genesis,GV70 3.5T Sport,2023,5400.0,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Green,Beige,not_reported,No,60000.0,False,1,5400.000000,375.0,3.5,107.142857,False


In [ ]:
#
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
#
x_train = cleaned_train_df.drop(columns=["default"])
y_train = cleaned_train_df["default"]

x_test = cleaned_test_df.drop(columns=["default"])
y_test = cleaned_test_df["default"]

In [ ]:
#
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las demas variables al intervalo [0, 1].
# - Selecciona las K mejores caracteristicas.
# - Ajusta un modelo de regresion logistica.
#
from sklearn.compose import ColumnTransformer  # type: ignore
from sklearn.linear_model import LogisticRegression  # type: ignore
from sklearn.pipeline import Pipeline  # type: ignore
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler  # type: ignore
from sklearn.feature_selection import SelectKBest, f_classif  # type: ignore


# Columnas categoricas:
#        SEX: Genero (1=male; 2=female).
#  EDUCATION: Educacion (0=N/A; 1=graduate school; 2=university; 3=high school; 4=others).
#   MARRIAGE: Estado civil (0=N/A; 1=married; 2=single; 3=others).
pipeline = Pipeline(
    [
        (
            "transformer",
            ColumnTransformer(
                [
                    (
                        "encoder",
                        OneHotEncoder(),
                        ["SEX", "EDUCATION", "MARRIAGE"],
                    ),
                ],
                remainder=MinMaxScaler(),
            ),
        ),
        ("selectkbest", SelectKBest(f_classif)),
        ("model", LogisticRegression(max_iter=1000)),
    ]
)

In [ ]:
#
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
#
import warnings
from sklearn.model_selection import GridSearchCV  # type: ignore

warnings.filterwarnings("ignore")

param_grid = {
    "model__C": [0.6],
    "selectkbest__k": [1],
}

grid_search_pipeline = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=10,
    scoring="balanced_accuracy",
    n_jobs=-1,
)

grid_search_pipeline.fit(x_train, y_train)

print(grid_search_pipeline.best_estimator_)
print(grid_search_pipeline.score(x_train, y_train))
print(grid_search_pipeline.score(x_test, y_test))

In [ ]:
#
# Paso 5.
# Salve el modelo como "files/models/model.pkl".
#
import os
import pickle

if not os.path.exists("../files/models"):
    os.makedirs("../files/models")

with open("../files/models/model.pkl", "wb") as file:
    pickle.dump(grid_search_pipeline, file)

In [ ]:
#
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'type': 'metrics', 'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
#
import os

from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score  # type: ignore

if not os.path.exists("../files/output"):
    os.makedirs("../files/output")

metrics = {
    "type": "metrics",
    "dataset": "train",
    "precision": float(precision_score(y_train, grid_search_pipeline.predict(x_train))),
    "balanced_accuracy": float(
        balanced_accuracy_score(y_train, grid_search_pipeline.predict(x_train))
    ),
    "recall": float(recall_score(y_train, grid_search_pipeline.predict(x_train))),
    "f1_score": float(f1_score(y_train, grid_search_pipeline.predict(x_train))),
}

with open("../files/output/metrics.json", "w") as file:
    file.write(str(metrics).replace("'", '"'))
    file.write("\n")

display(metrics)

metrics = {
    "type": "metrics",
    "dataset": "test",
    "precision": float(precision_score(y_test, grid_search_pipeline.predict(x_test))),
    "balanced_accuracy": float(
        balanced_accuracy_score(y_test, grid_search_pipeline.predict(x_test))
    ),
    "recall": float(recall_score(y_test, grid_search_pipeline.predict(x_test))),
    "f1_score": float(f1_score(y_test, grid_search_pipeline.predict(x_test))),
}

with open("../files/output/metrics.json", "a") as file:
    file.write(str(metrics).replace("'", '"'))
    file.write("\n")

display(metrics)

In [ ]:
#
# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

from sklearn.metrics import confusion_matrix


cm_train = pd.DataFrame(
    data=confusion_matrix(y_train, grid_search_pipeline.predict(x_train)),
    index=["True 0", "True 1"],
    columns=["Predicted 0", "Predicted 1"],
)

metrics = {
    "type": "cm_matrix",
    "dataset": "train",
    "true_0": {
        "predicted_0": int(cm_train.loc["True 0", "Predicted 0"]),
        "predicted_1": int(cm_train.loc["True 0", "Predicted 1"]),
    },
    "true_1": {
        "predicted_0": int(cm_train.loc["True 1", "Predicted 0"]),
        "predicted_1": int(cm_train.loc["True 1", "Predicted 1"]),
    },
}

with open("../files/output/metrics.json", "a") as file:
    file.write(str(metrics).replace("'", '"'))
    file.write("\n")

display(metrics)

cm_test = pd.DataFrame(
    data=confusion_matrix(y_test, grid_search_pipeline.predict(x_test)),
    index=["True 0", "True 1"],
    columns=["Predicted 0", "Predicted 1"],
)

metrics = {
    "type": "cm_matrix",
    "dataset": "test",
    "true_0": {
        "predicted_0": int(cm_test.loc["True 0", "Predicted 0"]),
        "predicted_1": int(cm_test.loc["True 0", "Predicted 1"]),
    },
    "true_1": {
        "predicted_0": int(cm_test.loc["True 1", "Predicted 0"]),
        "predicted_1": int(cm_test.loc["True 1", "Predicted 1"]),
    },
}

with open("../files/output/metrics.json", "a") as file:
    file.write(str(metrics).replace("'", '"'))
    file.write("\n")

display(metrics)